In [107]:
import gurobi as gp
from gurobi import GRB
import numpy as np

Consider the following convex constrained optimization problem:
$$
\min f(x)=x_1^2 + 2x_2^2
$$
subject to
$$
\begin{aligned}
x_1 + x_2 &= 3 \\
x_1 &\geq 0\\
x_2 &\geq 0
\end{aligned}
$$
This file tries to use the Lagrangian multiplier method, augmented Lagrangian multiplier method, and ADMM to solve the problem.

In [108]:
# Use Gurobi to solve the following problem
model = gp.Model()
x1 = model.addVar(lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x1")
x2 = model.addVar(lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x2")
model.setObjective(x1**2 + 2*x2**2, GRB.MINIMIZE)
model.addConstr(x1 + x2 == 3)
model.setParam("OutputFlag", 0)
model.optimize()
print("x1 = ", x1.x)
print("x2 = ", x2.x)
print("obj = ", model.objVal)

x1 =  1.9999999994655229
x2 =  1.000000000534477
obj =  5.999999999999999


In [109]:
# functions
def obj_func(x1, x2):
    """
    Objective function.
    """
    return x1**2 + 2*x2**2

def constraint(x1, x2):
    """
    Constraint function.
    """
    return x1 + x2 - 3

def Lagrangian(x1, x2, lam):
    """
    Lagrangian function.
    """
    return x1**2 + 2*x2**2 + lam*(x1 + x2 - 3)

def dLdx1(x1, x2, lam):
    """
    Derivative of Lagrangian function with respect to x1.
    """
    return 2*x1 + lam

def dLdx2(x1, x2, lam):
    """
    Derivative of Lagrangian function with respect to x2.
    """
    return 4*x2 + lam

def dLdlam(x1, x2, lam):
    """
    Derivative of Lagrangian function with respect to lambda.
    """
    return x1 + x2 - 3

def min_x1_x2(lam):
    """
    Given lambda, return the optimal x1 and x2.
    """
    x1 = -lam/2
    x2 = -lam/4
    return x1, x2

def dual_func(lam):
    """
    Dual function.
    """
    x1, x2 = min_x1_x2(lam)
    return obj_func(x1, x2)

In [110]:
# Method of Lagrange multipliers
# Initialize
x1 = 0
x2 = 0
lam = 0
# Iteration
for i in range(100):
    x1, x2 = min_x1_x2(lam)
    lam = lam + 0.1 * dLdlam(x1, x2, lam) # use MSA to update lambda
    print(f"iter = {i}, x1 = {x1:e}, x2 = {x2:e}, lam = {lam:e}, LB = {dual_func(lam):e}, constraint={constraint(x1, x2):e}")

iter = 0, x1 = 0.000000e+00, x2 = 0.000000e+00, lam = -3.000000e-01, LB = 3.375000e-02, constraint=-3.000000e+00
iter = 1, x1 = 1.500000e-01, x2 = 7.500000e-02, lam = -5.775000e-01, LB = 1.250648e-01, constraint=-2.775000e+00
iter = 2, x1 = 2.887500e-01, x2 = 1.443750e-01, lam = -8.341875e-01, LB = 2.609508e-01, constraint=-2.566875e+00
iter = 3, x1 = 4.170938e-01, x2 = 2.085469e-01, lam = -1.071623e+00, LB = 4.306413e-01, constraint=-2.374359e+00
iter = 4, x1 = 5.358117e-01, x2 = 2.679059e-01, lam = -1.291252e+00, LB = 6.252491e-01, constraint=-2.196282e+00
iter = 5, x1 = 6.456258e-01, x2 = 3.228129e-01, lam = -1.494408e+00, LB = 8.374705e-01, constraint=-2.031561e+00
iter = 6, x1 = 7.472039e-01, x2 = 3.736020e-01, lam = -1.682327e+00, LB = 1.061334e+00, constraint=-1.879194e+00
iter = 7, x1 = 8.411636e-01, x2 = 4.205818e-01, lam = -1.856153e+00, LB = 1.291989e+00, constraint=-1.738255e+00
iter = 8, x1 = 9.280763e-01, x2 = 4.640382e-01, lam = -2.016941e+00, LB = 1.525519e+00, constrai

In [111]:
def augmented_Lagrangian(x1, x2, lam, rho):
    """
    Augmented Lagrangian function.
    """
    return x1**2 + 2*x2**2 + lam*(x1 + x2 - 3) + rho/2*(x1 + x2 - 3)**2

def dALdx1(x1, x2, lam, rho):
    """
    Derivative of augmented Lagrangian function with respect to x1.
    """
    return 2*x1 + lam + rho*(x1 + x2 - 3)

def dALdx2(x1, x2, lam, rho):
    """
    Derivative of augmented Lagrangian function with respect to x2.
    """
    return 4*x2 + lam + rho*(x1 + x2 - 3)

def dALdlam(x1, x2, lam, rho):
    """
    Derivative of augmented Lagrangian function with respect to lambda.
    """
    return x1 + x2 - 3

def min_x1_x2_AL(lam, rho):
    """
    Given lambda and rho, return the optimal x1 and x2.
    """
    A = np.array([[2 + rho, rho], [rho, 4 + rho]])
    b = np.array([3*rho - lam, 3*rho-lam])
    x1, x2 = np.linalg.solve(A, b)
    return x1, x2

In [112]:
# Augmented Lagrangian method
# Initialize
x1 = 0
x2 = 0
lam = 0
rho = 1
eta = 1.1
# Iteration
for i in range(30):
    x1, x2 = min_x1_x2_AL(lam, rho)
    lam = lam + rho * constraint(x1, x2)
    rho = rho * eta
    print(f"iter = {i}, x1 = {x1:e}, x2 = {x2:e}, lam = {lam:e}, LB = {dual_func(lam):e}, constraint={constraint(x1, x2):e}")

iter = 0, x1 = 8.571429e-01, x2 = 4.285714e-01, lam = -1.714286e+00, LB = 1.102041e+00, constraint=-1.714286e+00
iter = 1, x1 = 1.373777e+00, x2 = 6.868885e-01, lam = -2.747554e+00, LB = 2.830894e+00, constraint=-9.393346e-01
iter = 2, x1 = 1.671705e+00, x2 = 8.358524e-01, lam = -3.343410e+00, LB = 4.191895e+00, constraint=-4.924428e-01
iter = 3, x1 = 1.835709e+00, x2 = 9.178543e-01, lam = -3.671417e+00, LB = 5.054739e+00, constraint=-2.464370e-01
iter = 4, x1 = 1.921694e+00, x2 = 9.608471e-01, lam = -3.843388e+00, LB = 5.539363e+00, constraint=-1.174586e-01
iter = 5, x1 = 1.964534e+00, x2 = 9.822668e-01, lam = -3.929067e+00, LB = 5.789088e+00, constraint=-5.319968e-02
iter = 6, x1 = 1.984770e+00, x2 = 9.923848e-01, lam = -3.969539e+00, LB = 5.908966e+00, constraint=-2.284551e-02
iter = 7, x1 = 1.993813e+00, x2 = 9.969063e-01, lam = -3.987625e+00, LB = 5.962933e+00, constraint=-9.280992e-03
iter = 8, x1 = 1.997627e+00, x2 = 9.988136e-01, lam = -3.995255e+00, LB = 5.985772e+00, constrai

In [113]:
def min_x1_AL(x2, lam, rho):
    """
    Given x2, lambda, and rho, return the optimal x1.
    """
    return (3*rho - lam - rho*x2)/(2 + rho)

def min_x2_AL(x1, lam, rho):
    """
    Given x1, lambda, and rho, return the optimal x2.
    """
    return (3*rho - lam - rho*x1)/(4 + rho)

In [114]:
# Alternating direction method of multipliers (ADMM)
# Initialize
x1 = 0
x2 = 0
lam = 0
rho = 1
eta = 1.1
# Iteration
for i in range(30):
    x1 = min_x1_AL(x2, lam, rho)
    x2 = min_x2_AL(x1, lam, rho)
    lam = lam + rho * constraint(x1, x2)
    rho = rho * eta
    print(f"iter = {i}, x1 = {x1:e}, x2 = {x2:e}, lam = {lam:e}, LB = {dual_func(lam):e}, constraint={constraint(x1, x2):e}")

iter = 0, x1 = 1.000000e+00, x2 = 4.000000e-01, lam = -1.600000e+00, LB = 9.600000e-01, constraint=-1.600000e+00
iter = 1, x1 = 1.438710e+00, x2 = 6.504744e-01, lam = -2.601898e+00, LB = 2.538702e+00, constraint=-9.108159e-01
iter = 2, x1 = 1.696207e+00, x2 = 8.022049e-01, lam = -3.208820e+00, LB = 3.861196e+00, constraint=-5.015884e-01
iter = 3, x1 = 1.841515e+00, x2 = 8.911581e-01, lam = -3.564632e+00, LB = 4.764976e+00, constraint=-2.673274e-01
iter = 4, x1 = 1.920322e+00, x2 = 9.416718e-01, lam = -3.766687e+00, LB = 5.320474e+00, constraint=-1.380061e-01
iter = 5, x1 = 1.961397e+00, x2 = 9.694960e-01, lam = -3.877984e+00, LB = 5.639535e+00, constraint=-6.910654e-02
iter = 6, x1 = 1.981977e+00, x2 = 9.843913e-01, lam = -3.937565e+00, LB = 5.814157e+00, constraint=-3.363211e-02
iter = 7, x1 = 1.991892e+00, x2 = 9.921607e-01, lam = -3.968643e+00, LB = 5.906297e+00, constraint=-1.594772e-02
iter = 8, x1 = 1.996488e+00, x2 = 9.961214e-01, lam = -3.984486e+00, LB = 5.953547e+00, constrai